In [1]:
#!pip install hyperopt

In [15]:
import pandas as pd
import numpy as np
import os
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
from skimage import color, exposure

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard
sns.set()

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.fit(
        X_train,
        y_train,
        batch_size = params_fit.get('batch_size', 128),
        epochs = params_fit.get('epochs', 5),
        verbose = params_fit.get('verbose', 1),
        validation_data = params_fit.get('validation_data', (X_train, y_train)),
        callbacks = [tensorboard_callback]
    )
    return model

def predict(model_trained, X_test, y_test, scoring = accuracy_score):

    y_test_norm = np.argmax(y_test, axis=1)

    y_pred_prob = model_trained.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
    return Sequential([
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(0.3),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(0.3),

        Flatten(),

        Dense(1024, activation='relu'),
        Dropout(0.3),
        
        Dense(1024, activation='relu'),
        Dropout(0.3),

        Dense(num_classes, activation='softmax')
    ])

In [9]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 16ms/step - loss: 2.7551 - accuracy: 0.2485 - val_loss: 1.3728 - val_accuracy: 0.5729
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 0.9660 - accuracy: 0.6949 - val_loss: 0.1973 - val_accuracy: 0.9432
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3623 - accuracy: 0.8909 - val_loss: 0.0793 - val_accuracy: 0.9746
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2191 - accuracy: 0.9348 - val_loss: 0.0461 - val_accuracy: 0.9874
Epoch 5/5
272/272 [==============================] - 4s 14ms/step - loss: 0.1699 - accuracy: 0.9507 - val_loss: 0.0309 - val_accuracy: 0.9905


0.9673469387755103

In [11]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.2045 - accuracy: 0.9673


0.9673469662666321

In [0]:
def get_model(params):
    return Sequential([
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_one']),

        Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_two']),

        Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
        Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_three']),

        Flatten(),

        Dense(1024, activation='relu'),
        Dropout(params['dropout_dense_block_one']),
        
        Dense(1024, activation='relu'),
        Dropout(params['dropout_dense_block_two']),

        Dense(num_classes, activation='softmax')
    ])

In [0]:
def func_obj(params):
    model = get_model(params)
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    model.fit(
        X_train,
        y_train,
        batch_size = int(params.get('batch_size', 128)),
        epochs = 5,
        verbose = 0,

    )
    score = model.evaluate(X_test, y_test, verbose=0)
    accuracy = score[1]
    print(params, 'accuracy={}'.format(accuracy))

    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [33]:
space ={
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 160.0, 'dropout_cnn_block_one': 0.4149155835107487, 'dropout_cnn_block_three': 0.35917357015639784, 'dropout_cnn_block_two': 0.35374932276915233, 'dropout_dense_block_one': 0.46413745398773615, 'dropout_dense_block_two': 0.5046169125380096}
accuracy=0.9532879590988159
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.32074603051992623, 'dropout_cnn_block_three': 0.47024758343267403, 'dropout_cnn_block_two': 0.36123356017401853, 'dropout_dense_block_one': 0.5247458359215679, 'dropout_dense_block_two': 0.3577838494162843}
accuracy=0.9589568972587585
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.431766000601564, 'dropout_cnn_block_three': 0.33520349298391766, 'dropout_cnn_block_two': 0.42876209003776644, 'dropout_dense_block_one': 0.575679339242251, 'dropout_dense_block_two': 0.5313029794941687}
accuracy=0.8659864068031311
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.40878354935639305, 'dropout_cnn_block_three': 0.3302670823275225, 'dropout_cnn_block_two': 0.381006300